<a href="https://www.nvidia.com/dli"> <img src="../images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 1.0 데이터 살펴보기

이 노트북에서는 과정의 이 섹션의 프로젝트에 대한 데이터세트 및 어노테이션(주석)을 살펴봅니다.  데이터를 자세히 살펴보는 이유는 향후 프로젝트를 위해 자체 데이터세트를 구조화할 수 있는 방법에 대한 인사이트를 제공할 뿐 아니라 모델에 대한 입력의 명확한 그림을 제시하기 위한 것입니다.

**[1.1 Corpus Annotated Data](#1.1-Corpus-Annotated-Data)<br>**
**[1.2 텍스트 분류 데이터세트](#1.2-텍스트-분류-데이터세트)<br>**
&nbsp;&nbsp;&nbsp;&nbsp;[1.2.1 연습 : 테스트 세트 살펴보기](#1.2.1-연습-:-테스트-세트-살펴보기)<br>
**[1.3 NER 데이터세트](#1.3-NER-데이터세트)<br>**

---
# 1.1 Corpus Annotated Data

[NCBI-질병 코퍼스 언어 자료](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)는 14명의 주석자가 어노테이션을 진행 한 단 793개의 PubMed Abstract 세트입니다. 이 어노테이션은 명확하게 정의된 규칙을 사용해 Abstract 텍스트에 삽입되는 HTML 스타일 태그의 형식을 지닙니다.  어노테이션은 명명된 질병을 식별하고 향후 Abstract에서 언급되는 질병을 식별할 수 있도록 언어 모델을 미세 조정하는 데 사용될 수 있습니다. *그러한 질병이 원래 트레이닝 세트의 일부였는지 여부는 상관없습니다*.  

다음은 언어 자료의 레이블링이 달린 Abstract을 보여주는 예입니다.

```html
10021369	Identification of APC2, a homologue of the <category="Modifier">adenomatous polyposis coli tumour</category> suppressor .	The <category="Modifier">adenomatous polyposis coli ( APC ) tumour</category>-suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta ( GSK-3beta ) , axin / conductin and betacatenin . Complex formation induces the rapid degradation of betacatenin . In <category="Modifier">colon carcinoma</category> cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf-4 transcription factor ( reviewed in [ 1 ] [ 2 ] ) . Here , we report the identification and genomic structure of APC homologues . Mammalian APC2 , which closely resembles APC in overall domain structure , was functionally analyzed and shown to contain two SAMP domains , both of which are required for binding to conductin . Like APC , APC2 regulates the formation of active betacatenin-Tcf complexes , as demonstrated using transient transcriptional activation assays in APC - / - <category="Modifier">colon carcinoma</category> cells . Human APC2 maps to chromosome 19p13 . 3 . APC and APC2 may therefore have comparable functions in development and <category="SpecificDisease">cancer</category> .
```

이 예에서는 Abtract 내에 다음 태그가 표시됩니다.:
```html
<category="Modifier">adenomatous polyposis coli tumour</category>
<category="Modifier">adenomatous polyposis coli ( APC ) tumour</category>
<category="Modifier">colon carcinoma</category>
<category="Modifier">colon carcinoma</category>
<category="SpecificDisease">cancer</category>
```
이 문서의 목적에 따라 식별된 범주(예: "Modifier", "Specific Disease" 및 기타 몇 가지 질병)를 일반적으로 "질병"으로 간주하겠습니다.  더 많은 예를 보려면, 왼쪽의 파일 탐색기를 사용하여 코퍼스의 텍스트를 탐색하거나 직접 파일을 열면 됩니다. 

* [data/NCBI_corpus/NCBI_corpus_training.txt](data/NCBI_corpus/NCBI_corpus_training.txt)
* [data/NCBI_corpus/NCBI_corpus_testing.txt](data/NCBI_corpus/NCBI_corpus_testing.txt)
* [data/NCBI_corpus/NCBI_corpus_development.txt](data/NCBI_corpus/NCBI_corpus_development.txt)

이 코퍼스에서 2개의 데이터세트인 텍스트 분류 데이터세트와 NER(명명된 엔터티 인식) 데이터세트가 파생됩니다.  텍스트 분류 데이터세트는 Abstract에서 3개의 광범위한 질병 그룹화 간에 레이블을 지정합니다.  이 간단한 분할을 사용하여 NLP 텍스트 분류 작업을 보여드리겠습니다.   NER 데이터세트는 개별 단어에 질병 레이블을 지정합니다.  이 데이터세트는 NLP NER 작업에 사용됩니다.

---
# 1.2 텍스트 분류 데이터세트

텍스트 분류 작업은 텍스트의 내용에 따라 텍스트를 분류하고자 합니다.  텍스트 분류의 적용 예로는 감정 분석(2개의 클래스)과 주제 레이블 지정(여러 클래스)이 있습니다.  필요한 데이터세트의 종류를 이해하려면 먼저 하고 싶은 질문이 무엇인지 결정해야 합니다.


### 감정 분석 - Sentiment Analysis
예를 들어 영화 리뷰를 분석할 경우 다음과 같이 질문할 수 있습니다.:<br>
**주어진 영화 리뷰 문장에서 감정이 긍정적인가 아니면 부정적인가?**<br>
이런 분석에서는 문장을 살펴봐야 하며 클래스는 "긍정적"과 "부정적" 두 가지뿐입니다.  트레이닝 세트의 각 문장에 이 두 가지 중 하나로 레이블을 지정해야 합니다. 감정 분석은 기업들이 온라인 대화와 피드백에서 제품, 브랜드 또는 서비스에 대한 고객의 감정을 파악하는 데 널리 사용됩니다.

### 멀티 클래스 분석 - Multi-Class Analysis
멀티 클래스 텍스트 분류 프로젝트에서는 다른 질문을 할 것입니다.:<br>
**주어진 의학 질병 Abstract은 암 또는 신경계 질환, 아니면 그 밖의 다른 질병에 대한 Abstract인가?**<br>
해당 사용 사례를 위해 문장이 아닌 전체 Abstract을 살펴보고 "암", "신경계 질환" 및 "기타"의 3가지 클래스를 식별했습니다.  이 실습 랩에서는 단순한 접근 방식으로 이러한 3가지 범주에 속하는 식별된 질병을 기반으로 Abstract에 레이블이 지정됩니다.  데이터는 `.tsv` 형식으로 저장됩니다.  이는 일반적인 `.csv` 쉼표로 구분된 형식과 유사하지만 쉼표 대신 탭을 사용해 열을 구분합니다.  다음 셀을 실행하면 텍스트 분류를 위한 3가지 클래스 데이터세트의 `.tsv` 파일 목록이 표시됩니다.

In [ ]:
TC_DATA_DIR = '/dli/task/data/NCBI_tc-3/'
!ls -lh $TC_DATA_DIR

JupyterLab에서는 왼쪽의 파일 탐색기를 사용해 파일과 데이터를 탐색할 수 있습니다.  노트북에서는 [_pandas_](https://pandas.pydata.org/docs/user_guide/index.html)를 사용해 데이터를 가져와서 보겠습니다. 이는 모델의 데이터를 가져오는 유용한 방법입니다.  

`pd.read_csv()` 함수를 사용해 탭을 구분 기호로 지정하여 _pandas_ DataFrame 개체에 데이터를 가져올 수 있습니다.  `.head()` 함수는 상위 5개의 데이터 행을 표시합니다.  각 행에는 소문자 Abstract과 레이블이 포함되어 있습니다.  "cancer (암)", "neurological (신경계 질환)", "other (기타)"의 세 범주의 레이블 값은 각각 0, 1, 2입니다.


In [ ]:
import pandas as pd
pd.options.display.max_colwidth = None

In [ ]:
train_df = pd.read_csv(TC_DATA_DIR + 'train.tsv', sep='\t')
train_df.head()

## 1.2.1 연습 : 테스트 세트 살펴보기
다음 셀에 있는 테스트 세트에 동일한 작업을 수행해 보십시오. 실습 수행에 어려움이 있는 경우, [솔루션](solutions/ex1.2.1.ipynb) 파일을 참고해 주세요.

In [ ]:
# Change the FIXME lines to view the test set.
# ???_df = None #FIXME
# ???_df.head() #FIXME

아주 비슷한 예를 볼 수 있을 것입니다. 테스트 샘플은 최종 평가시 활용될 예정입니다. 

---
# 1.3 NER 데이터세트

NER 작업에서는 새로운 질문을 할 것입니다.<br>
**의학 Abstract의 주어진 문장에서 어떤 질병이 언급되었나요?**<br>
이 경우 데이터 입력은 Abstract의 문장이며 레이블은 명명된 질병 엔터티의 위치입니다.  데이터세트에 대해 제공된 정보를 살펴보십시오.


In [ ]:
NER_DATA_DIR = '/dli/task/data/NCBI_ner-3/'
!ls -lh $NER_DATA_DIR

NER 작업을 수행하려면 두 개의 파일이 필요 합니다. : 텍스트 문장과 레이블 파일. 다음 2개의 셀을 실행하여 이 두 파일의 샘플을 확인합니다.

In [ ]:
!head $NER_DATA_DIR/text_train.txt

In [ ]:
!head $NER_DATA_DIR/labels_train.txt

### IOB 태그 지정
Abstract이 문장들로 나뉘어진 것을 볼 수 있습니다.  그런 다음 각 문장이 코퍼스의 원래 HTML 스타일 태그에 해당하는 레이블이 있는 단어로 추가로 구문 분석됩니다. 

NER 데이터세트의 문장과 레이블이 _Inside, Outside, Beginning (IOB)_ 태그 지정으로 서로 매핑됩니다. 공백으로 구분된 모든 부분은 구두점을 포함한 단어입니다.  첫 번째 문장의 경우 다음과 같이 매핑됩니다.

```text
Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
O              O  O    O O O         O  O   B           I         I    I      O          O  
```

원래 코퍼스 태그로 재호출하면 다음과 같습니다:

```html
Identification of APC2, a homologue of the <category="Modifier">adenomatous polyposis coli tumour</category> suppressor .
```
태그가 지정된 텍스트의 시작어 "선종성 (adenomatous)"에 이제 <span style="font-family:verdana;font-size:110%;">B</span>(Beginning) 태그로 IOB 태그가 지정되고, 질병의 다른 부분인 "대장용종증 (polyposis coli tumour)"에는 <span style="font-family:verdana;font-size:110%;">I</span>(Inside) 태그가 지정되고, 나머지 모든 부분에는 <span style="font-family:verdana;font-size:110%;">O</span>(Outside) 태그가 지정됩니다.



---
<h2 style="color:green;">축하합니다!</h2>

여러분은 텍스트 분류 및 NER 작업에 대한 데이터세트를 살펴보았으며 다음과 같은 내용을 배웠습니다.
* 텍스트 분류 트레이닝 데이터에는 텍스트 내용에 범주를 매핑하는 레이블이 있음
* NER 트레이닝 데이터는 단어를 I, O, B(inside, outside, beginning)와 같은 태그에 매핑하여 엔터티를 식별함

다음으로는, NVIDIA NeMo 툴킷 기능 몇 가지와 NeMo를 사용해 NLP 작업을 설정 및 실행하는 방법을 간단히 살펴보겠습니다.<br>

[2.0 텍스트 분류자(Text Classifier) 구축하기(NVIDIA NeMo v1.0)](020_TextClassification.ipynb)로 이동하십시오.



<a href="https://www.nvidia.com/dli"> <img src="../images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>